## Задание 2. Проверить насколько хорошо работает NER
данные брать из http://www.labinform.ru/pub/named_entities/
1. взять нер из nltk
2. написать свой нер попробовать разные подходы
* передаём в сетку токен и его соседей
* передаём в сетку только токен
3. сделать выводы по вашим экспериментам какой из подходов успешнее справляется


In [1]:
from corus import load_ne5
import re
import nltk
import spacy
from spacy import displacy
from spacy.lang.ru.examples import sentences 
import en_core_web_sm

In [2]:
# !wget http://www.labinform.ru/pub/named_entities/collection5.zip
# !unzip collection5.zip
# !rm collection5.zip

# !pip install corus

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

In [3]:
dir = 'Collection5/'
records = load_ne5(dir)
document = next(records).text
nltk.pos_tag(nltk.word_tokenize(document))[:10]

[('Собянин', 'JJ'),
 ('собрал', 'NNP'),
 ('необходимые', 'NNP'),
 ('подписи', 'NNP'),
 ('избирателей', 'NNP'),
 ('в', 'NNP'),
 ('свою', 'NNP'),
 ('поддержку', 'NNP'),
 ('ОНФ', 'NNP'),
 ('не', 'NNP')]

In [4]:
list({(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') })[:10]

[('Собянин', 'PERSON'),
 ('Исаева', 'PERSON'),
 ('Сергей Железняк', 'PERSON'),
 ('Андрея Исаева', 'PERSON'),
 ('Сергей Нарышкин', 'PERSON'),
 ('Евгеньевич', 'ORGANIZATION'),
 ('ОНФ', 'ORGANIZATION'),
 ('Нарышкин', 'ORGANIZATION'),
 ('Госдумы', 'PERSON'),
 ('Галушка', 'PERSON')]

*****************************************************

In [5]:
# !pip  install razdel
from razdel import tokenize

In [6]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [7]:
import pandas as pd
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])
df_words['tag'].value_counts()

OUT         218941
PER          21175
ORG          13630
LOC           4568
GEOPOLIT      4356
MEDIA         2480
Name: tag, dtype: int64

In [8]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [9]:
from sklearn import model_selection, preprocessing, linear_model
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])
# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [10]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))
train_data = train_data.batch(256)
valid_data = valid_data.batch(256)

In [11]:
AUTOTUNE = tf.data.AUTOTUNE
train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [12]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    #ngrams=(1, 3),
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [13]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.dr1 = Dropout(0.5)
        self.fc2 = Dense(100, activation='relu')
        self.dr2 = Dropout(0.25)
        self.fc3 = Dense(50, activation='relu')
        self.fc4 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.dr1(fc_x)
        fc_x = self.fc2(fc_x)
        fc_x = self.dr2(fc_x)
        fc_x = self.fc3(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc4(concat_x)
        return prob

In [14]:
mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
777/777 [==============================] - 24s 29ms/step - loss: 0.4147 - accuracy: 0.8751 - val_loss: 0.2462 - val_accuracy: 0.9288
Epoch 2/3
777/777 [==============================] - 22s 28ms/step - loss: 0.1630 - accuracy: 0.9512 - val_loss: 0.2421 - val_accuracy: 0.9388
Epoch 3/3
777/777 [==============================] - 21s 27ms/step - loss: 0.1236 - accuracy: 0.9611 - val_loss: 0.2337 - val_accuracy: 0.9405


In [15]:
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    ngrams=(1, 3),
    output_sequence_length=seq_len)

In [16]:
mmodel2 = modelNER()
mmodel2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel2.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
777/777 [==============================] - 22s 28ms/step - loss: 0.7244 - accuracy: 0.8233 - val_loss: 0.6951 - val_accuracy: 0.8259
Epoch 2/3
777/777 [==============================] - 21s 27ms/step - loss: 0.6976 - accuracy: 0.8257 - val_loss: 0.6934 - val_accuracy: 0.8259
Epoch 3/3
777/777 [==============================] - 21s 28ms/step - loss: 0.6952 - accuracy: 0.8257 - val_loss: 0.6932 - val_accuracy: 0.8259


**Вывод:** Подача в нейронную сети ngram показало меньшую точность, по сравнению с подачей отдельных токенов